# Settle Up Problem (Dlužníček)

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [5]:
!pip install gurobipy
P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values())/len(c)  # the settlement value

Defaulting to user installation because normal site-packages is not writeable


Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x, y, z)$ representing the transactions: person $x$ should pay person $y$ amount of $z$ euros. The number of transactions (i.e., the length of the list) should be minimized.

For the given instance, the **optimal solution** consists of 3 transactions, namely

```
A -> B: 250.0
C -> D: 50.0
C -> B: 90.0
```

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [11]:
import gurobipy as g  # import Gurobi module

P = set(["A", "B", "C", "D"])
c = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
sv = sum(c.values()) / len(c)  # the settlement value

# model --------------------------------------------------
m = g.Model()

print(c)

# - ADD VARIABLES
t_a = m.addVars(len(c), len(c), vtype=g.GRB.INTEGER, lb=0)  # velikost transakce
t_s = m.addVars(len(c), len(c), vtype=g.GRB.BINARY, lb=0)  # probla transakce

M = sum(c.values())  # Fix: Correct summation of dictionary values

# - ADD CONSTRAINTS
for person_id in range(len(c)):
    key = list(c.keys())[person_id]  # Fix: Correct indexing
    m.addConstr(t_a.sum(person_id, "*") - t_a.sum("*", person_id) + c[key] == sv)

for i in range(len(c)):
    for j in range(len(c)):
        m.addConstr(t_a[i, j] <= M * t_s[i, j])  # Fix: Correct dictionary-like access

# - SET OBJECTIVE
# chceme minimalizovat pocet transakci
m.setObjective(t_s.sum())

# call the solver -------------------------------------------
m.optimize()

# print the solution -----------------------------------------
print('\nSOLUTION:')
print([t_a[i, j].x for i in range(len(c)) for j in range(len(c))])


{'A': 0, 'B': 590, 'C': 110, 'D': 300}
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 20 rows, 32 columns and 56 nonzeros
Model fingerprint: 0x58fc8560
Variable types: 0 continuous, 32 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 3.0000000
Presolve removed 4 rows and 8 columns
Presolve time: 0.01s
Presolved: 16 rows, 24 columns, 48 nonzeros
Variable types: 0 continuous, 24 integer (12 binary)

Root relaxation: objective 3.900000e-01, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   